In [ ]:
!pip install tensorflow-gpu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import os
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_path ='/content/drive/MyDrive/Seventh Semester Project/Datasets/Train'
valid_path = '/content/drive/MyDrive/Seventh Semester Project/Datasets/Validate'
test_path = '/content/drive/MyDrive/Seventh Semester Project/Datasets/Test'
Img_size = [224, 224]
batch_size = 512

In [ ]:
## Data Augmentation

train_generator =ImageDataGenerator(rescale = 1/255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   rotation_range=30,
                                   fill_mode='nearest')
valid_generator = ImageDataGenerator(rescale= 1/255)
test_generator = ImageDataGenerator(rescale = 1/ 255)

In [ ]:
# Dataset 

train_datset = train_generator.flow_from_directory(directory=train_path,
                                                   target_size= Img_size,
                                                   class_mode = 'sparse',
                                                   batch_size = batch_size,
                                                   color_mode = 'rgb', )

valid_datset = train_generator.flow_from_directory(directory=valid_path,
                                                   target_size= Img_size,
                                                   class_mode = 'sparse',
                                                   batch_size = batch_size,
                                                   color_mode = 'rgb', )

test_datset = train_generator.flow_from_directory(directory=test_path,
                                                   target_size= Img_size,
                                                   class_mode = 'sparse',
                                                   batch_size = batch_size,
                                                   color_mode = 'rgb', )

Found 10081 images belonging to 10 classes.
Found 2348 images belonging to 10 classes.
Found 2029 images belonging to 10 classes.


In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

In [ ]:
 base_model = InceptionResNetV2(include_top=False, input_shape=Img_size +[3], weights='imagenet')

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
model=Sequential()
model.add(base_model)
#model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(10,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 38400)             0         
                                                                 
 dense_1 (Dense)             (None, 225)               8640225   
                                                                 
Total params: 62,976,961
Trainable params: 8,640,225
Non-trainable params: 54,336,736
_________________________________________________________________


In [ ]:
opt=tf.keras.optimizers.Adam(lr=0.001)

model.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              optimizer=opt)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Here Early stopping callback is added for unnecessary running of epochs if model performance is poor 
filepath = '/content/drive/MyDrive/Seventh Semester Project/model/IRv2_base_model.h5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 10,
                              verbose = 1)

# for saving best model till execution
checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)

callback_list = [earlystopping, checkpoint]

In [ ]:
result = model.fit(train_datset, 
                   validation_data=valid_datset, 
                   epochs = 100,
                   callbacks = callback_list)
                    #verbose= 1)

Epoch 1/100
20/20 [==============================] - ETA: 0s - loss: 7.3675 - accuracy: 0.2855  
Epoch 00001: val_accuracy improved from -inf to 0.37819, saving model to /content/drive/MyDrive/Seventh Semester Project/model/IRv2_base_model.h5
20/20 [==============================] - 1981s 100s/step - loss: 7.3675 - accuracy: 0.2855 - val_loss: 3.5442 - val_accuracy: 0.3782
Epoch 2/100
20/20 [==============================] - ETA: 0s - loss: 2.3307 - accuracy: 0.5527 
Epoch 00002: val_accuracy improved from 0.37819 to 0.44591, saving model to /content/drive/MyDrive/Seventh Semester Project/model/IRv2_base_model.h5
20/20 [==============================] - 534s 27s/step - loss: 2.3307 - accuracy: 0.5527 - val_loss: 2.1240 - val_accuracy: 0.4459
Epoch 3/100
20/20 [==============================] - ETA: 0s - loss: 1.3720 - accuracy: 0.6092 
Epoch 00003: val_accuracy improved from 0.44591 to 0.48083, saving model to /content/drive/MyDrive/Seventh Semester Project/model/IRv2_base_model.h5
20/